In [1]:
import requests

In [2]:
googlePage = requests.get('https://www.google.com/')

In [17]:
HTMLString = googlePage.text
HTMLString = HTMLString.replace('<!DOCTYPE html>','')
HTMLString = HTMLString.replace('<!doctype html>','')

In [40]:
from bs4 import BeautifulSoup as chef

In [41]:
soup = chef(HTMLString, 'html.parser')

In [19]:
print(HTMLString)

<html itemscope="" itemtype="http://schema.org/WebPage" lang="ne"><head><meta content="text/html; charset=UTF-8" http-equiv="Content-Type"><meta content="/images/branding/googleg/1x/googleg_standard_color_128dp.png" itemprop="image"><title>Google</title><script nonce="Ntd6ym7DwfeUWFkoyy7YMw">(function(){var _g={kEI:'GbnIZuRp-tTV7w_xn4HwDw',kEXPI:'0,18167,3682099,173,942,8,89,302032,236535,2872,2891,43028,30022,16105,163859,2,39761,6699,41943,57740,2,2,1,6959,19673,8155,23351,7450,14985,9779,8213,54444,76209,15816,1804,21011,26071,1428,87,120,9708,3784,15784,14184,5225574,999,123,23,5991286,2841016,317,10,7,36,46,1,15,4,5,4,7,1,1,1,17,2,1,1,2,3,5,3,1,8,7,5,3,3,4,3,3,3,3,3,3,3,3,1,5,1,2,1,23936855,4043710,16672,43887,3,318,4,1281,3,2124363,23029351,12799,19311,81170,22623,15164,8181,154,10654,7069,4976,4,26573,21672,6752,155,2484,4400,9014,89,7736,7034,2105,4600,328,3217,4,1233,5,1766,1116,18557,410,518,4858,5,834,22,1670,5634,687,2730,3,3940,618,504,3,3015,2695,4479,5607,476,5784,682,56

In [43]:
for script_tag in soup.find_all('script'):
    script_tag.decompose()

In [50]:
# sample = """<html itemscope="" itemtype="http://schema.org/WebPage" lang="ne">"""
sample = str(soup)

In [ ]:
## loop through the HTML string handling multiple case
tag_record = []
tag_attribute_record = []
wait_for_ang_bracket_close = False
wait_for_whitespace_encounter = False
attribute_collection_flag = False
for i, ch in enumerate(HTMLString):
    if wait_for_ang_bracket_close:
        if ch == ' ':
            tag = ''.join(tag_record)
            tag_record = []
            wait_for_whitespace_encounter = False
            attribute_collection_flag = True
        else:
            tag_record.append(ch)
    if ch == '<':
        wait_for_ang_bracket_close=True
        wait_for_whitespace_encounter=True
    if ch == '>':
        attribute_collection_flag= False
        wait_for_ang_bracket_close=False
        wait_for_whitespace_encounter=False
        break


In [31]:
tag_record = []
tag_closure_record = []
attribute_record = []
tag_detected = False
closure_tag_detected = False
attribute_detected=False
tagStack = []
tagClosureStack = []
attributeStack = []
targetObj = {}
for i,ch in enumerate(sample):
    if ch == '<' and sample[i+1] != '/':
        tag_detected = True
        continue
    if ch == '<' and sample[i+1] == '/':
        closure_tag_detected = True
        continue
    if tag_detected and ch == '/' and sample[i+1] == '>':
        attribute_detected = False
        tag_detected = False
        attribute_record = []
        tag = ''.join(tag_record)
        tag_record = []
        tagStack.append(tag)
        tagClosureStack.append(tag)
    if tag_detected:
        if ch != " " or ch == '>':
            tag_record.append(ch)
        else:
            tag = ''.join(tag_record)
            tag_record = []
            tag_detected = False
            tagStack.append(tag)

            if ch == ' ':
                attribute_detected = True
                continue
    if attribute_detected:
        if ch != '>':
            attribute_record.append(ch)
        else:
            attribute_detected = False
            tag_detected = False
            attribute = ''.join(attribute_record)
            attribute_record = []
            attributeStack.append(attribute)
    if closure_tag_detected:
        if ch == '/':
            continue
        else:
            tag_closure_record.append(ch)
            if ch == '>':
                tag_closure = ''.join(tag_closure_record)
                tag_closure_record = []
                tagClosureStack.append(tag_closure)
                closure_tag_detected = False


In [51]:
print(sample)

<html itemscope="" itemtype="http://schema.org/WebPage" lang="ne"><head><meta content="text/html; charset=utf-8" http-equiv="Content-Type"/><meta content="/images/branding/googleg/1x/googleg_standard_color_128dp.png" itemprop="image"/><title>Google</title><style>#gbar,#guser{font-size:13px;padding-top:1px !important;}#gbar{height:22px}#guser{padding-bottom:7px !important;text-align:right}.gbh,.gbd{border-top:1px solid #c9d7f1;font-size:1px}.gbh{height:0;position:absolute;top:24px;width:100%}@media all{.gb1{height:22px;margin-right:.5em;vertical-align:top}#gbar{float:left}}a.gb1,a.gb4{text-decoration:underline !important}a.gb1,a.gb4{color:#00c !important}.gbi .gb4{color:#dd8e27 !important}.gbf .gb4{color:#900 !important}
</style><style>body,td,a,p,.h{font-family:arial,sans-serif}body{margin:0;overflow-y:scroll}#gog{padding:3px 8px 0}td{line-height:.8em}.gac_m td{line-height:17px}form{margin-bottom:20px}.h{color:#1967d2}em{font-weight:bold;font-style:normal}.lst{height:25px;width:496px}.

In [56]:
def parse_html(sample):
    tag_stack = []
    attribute_stack = []
    closure_stack = []
    content_stack = []

    tag_record = []
    attribute_record = []
    content_record = []
    attributes = {}

    i = 0
    while i < len(sample):
        ch = sample[i]

        # Detect the start of an opening tag
        if ch == '<' and i + 1 < len(sample) and sample[i + 1] != '/':
            tag_record = []
            i += 1
            while i < len(sample) and sample[i] not in [' ', '>', '/']:
                tag_record.append(sample[i])
                i += 1
            tag = ''.join(tag_record)
            tag_stack.append(tag)

            # Handle attributes within the tag
            attributes = {}
            while i < len(sample) and sample[i] != '>':
                if sample[i] == '/':  # Handle self-closing tags
                    closure_stack.append(tag)
                    break

                if sample[i] == ' ':
                    i += 1
                    attr_name = []
                    while i < len(sample) and sample[i] != '=' and sample[i] != '>':
                        attr_name.append(sample[i])
                        i += 1

                    if sample[i] == '=':
                        i += 1  # Skip '='
                        if sample[i] in ['"', "'"]:
                            quote = sample[i]
                            i += 1
                            attr_value = []
                            while i < len(sample) and sample[i] != quote:
                                attr_value.append(sample[i])
                                i += 1
                            attributes[''.join(attr_name).strip()] = ''.join(attr_value).strip()
                            i += 1  # Skip closing quote
                    else:
                        attributes[''.join(attr_name).strip()] = None
                else:
                    i += 1

            if sample[i] == '>':
                attribute_stack.append(attributes)

            if sample[i - 1] == '/':  # Self-closing tag
                closure_stack.append(tag)

        # Detect the start of a closing tag
        elif ch == '<' and i + 1 < len(sample) and sample[i + 1] == '/':
            i += 2
            closure_tag_record = []
            while i < len(sample) and sample[i] != '>':
                closure_tag_record.append(sample[i])
                i += 1
            closure_stack.append(''.join(closure_tag_record))

        # Detect content between tags
        elif ch != '<':
            content_record = []
            while i < len(sample) and sample[i] != '<':
                content_record.append(sample[i])
                i += 1
            content = ''.join(content_record).strip()
            if content:
                content_stack.append(content)
            i -= 1  # Step back to reprocess the '<'

        i += 1

    return tag_stack, attribute_stack, closure_stack, content_stack


tags, attributes, closures, contents = parse_html(sample)

print("Tags:", tags)
print("Attributes:", attributes)
print("Closures:", closures)
print("Contents:", contents)



Tags: ['html', 'head', 'meta', 'meta', 'title', 'style', 'style', 'body', 'div', 'div', 'nobr', 'b', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'u', 'div', 'nobr', 'span', 'span', 'span', 'a', 'a', 'a', 'div', 'div', 'center', 'br', 'div', 'img', 'br', 'br', 'form', 'table', 'tr', 'td', 'td', 'input', 'input', 'input', 'input', 'input', 'div', 'div', 'input', 'img', 'br', 'span', 'span', 'input', 'span', 'span', 'input', 'input', 'td', 'a', 'input', 'div', 'br', 'div', 'style', 'div', 'a', 'span', 'div', 'div', 'a', 'a', 'a', 'p', 'link']
Attributes: [{'itemscope': '', 'itemtype': 'http://schema.org/WebPage', 'lang': 'ne'}, {}, {}, {}, {}, {'bgcolor': '#fff'}, {'id': 'mngb'}, {'id': 'gbar'}, {}, {'class': 'gb1'}, {'class': 'gb1', 'href': 'https://www.google.com/imghp?hl=ne&amp;tab=wi'}, {'class': 'gb1', 'href': 'https://maps.google.com.np/maps?hl=ne&amp;tab=wl'}, {'class': 'gb1', 'href': 'https://play.google.com/?hl=ne&amp;tab=w8'}, {'class': 'gb1', 'href': 'https://news.google.com/?tab=w

In [59]:
len(tags), len(closures), len(contents), len(attributes)

(76, 76, 46, 56)

In [62]:
def parse_html_with_bs4(html_content):
    soup = chef(html_content, 'html.parser')

    tag_stack = []
    attribute_stack = []
    closure_stack = []
    content_stack = []

    # Traversing the tree
    for tag in soup.find_all(True):
        tag_stack.append(tag.name)
        attribute_stack.append(tag.attrs)
        
        # If the tag has content, we record the content
        if tag.string and tag.string.strip():
            content_stack.append(tag.string.strip())

        # Handle self-closing tags
        if tag.is_self_closing:
            closure_stack.append(tag.name)
        else:
            closure_stack.append(tag.name)

    return tag_stack, attribute_stack, closure_stack, content_stack


tags, attributes, closures, contents = parse_html_with_bs4(sample)

print("Tags:", tags)
print("Attributes:", attributes)
print("Closures:", closures)
print("Contents:", contents)


Tags: ['html', 'head', 'meta', 'meta', 'title', 'style', 'style', 'body', 'div', 'div', 'nobr', 'b', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'u', 'div', 'nobr', 'span', 'span', 'span', 'a', 'a', 'a', 'div', 'div', 'center', 'br', 'div', 'img', 'br', 'br', 'form', 'table', 'tr', 'td', 'td', 'input', 'input', 'input', 'input', 'input', 'div', 'div', 'input', 'img', 'br', 'span', 'span', 'input', 'span', 'span', 'input', 'input', 'td', 'a', 'input', 'div', 'br', 'div', 'style', 'div', 'a', 'span', 'div', 'div', 'a', 'a', 'a', 'p', 'link']
Attributes: [{'itemscope': '', 'itemtype': 'http://schema.org/WebPage', 'lang': 'ne'}, {}, {'content': 'text/html; charset=utf-8', 'http-equiv': 'Content-Type'}, {'content': '/images/branding/googleg/1x/googleg_standard_color_128dp.png', 'itemprop': 'image'}, {}, {}, {}, {'bgcolor': '#fff'}, {'id': 'mngb'}, {'id': 'gbar'}, {}, {'class': ['gb1']}, {'class': ['gb1'], 'href': 'https://www.google.com/imghp?hl=ne&tab=wi'}, {'class': ['gb1'], 'href': 'https://

In [63]:
len(tags), len(closures), len(contents), len(attributes)

(76, 76, 23, 76)

In [85]:
nicPage = requests.get('https://www.nicasiabank.com/tutorial/')
HTMLString = nicPage.text
HTMLString = HTMLString.replace('<!DOCTYPE html>','')
HTMLString = HTMLString.replace('<!doctype html>','')
NoScriptHTML = chef(HTMLString, 'html.parser')
for script_tag in NoScriptHTML.find_all('script'):
    script_tag.decompose()
HTMLString = str(NoScriptHTML)

In [86]:
def parse_html_with_bs4(html_content):
    soup = chef(html_content, 'html.parser')

    tag_stack = []
    attribute_stack = []
    closure_stack = []
    content_stack = []

    # Traversing the tree
    for tag in soup.find_all(True):
        # Add tag to tag stack
        tag_stack.append(tag.name)
        
        # Add attributes to attribute stack
        attribute_stack.append(tag.attrs)

        # Add content to content stack, assign empty string if no content
        if tag.string and tag.string.strip():
            content_stack.append(tag.string.strip())
        else:
            content_stack.append("")  # Empty string for tags without content

        # Add tag to closure stack (self-closing tags are handled similarly)
        closure_stack.append(tag.name)

    return tag_stack, attribute_stack, closure_stack, content_stack

tags, attributes, closures, contents = parse_html_with_bs4(HTMLString)

print("Tags:", tags)
print("Attributes:", attributes)
print("Closures:", closures)
print("Contents:", contents)


Tags: ['html', 'head', 'meta', 'link', 'meta', 'meta', 'meta', 'title', 'meta', 'meta', 'meta', 'link', 'meta', 'meta', 'meta', 'meta', 'meta', 'meta', 'meta', 'meta', 'meta', 'meta', 'meta', 'meta', 'meta', 'meta', 'link', 'link', 'link', 'link', 'link', 'noscript', 'body', 'div', 'main', 'nav', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'div', 'header', 'div', 'div', 'div', 'div', 'a', 'figure', 'button', 'svg', 'g', 'path', 'path', 'div', 'div', 'div', 'nav', 'ol', 'li', 'span', 'a', 'li', 'li', 'span', 'a', 'div', 'div', 'article', 'div', 'h1', 'h6', 'p', 'br', 'div', 'div', 'span', 'button', 'h6', 'svg', 'path', 'path', 'div', 'img', 'div', 'div', 'div', 'div', 'div', 'div', 'div', 'button', 'svg', 'line', 'polyline', 'div', 'div', 'button', 'div', 'div', 'button', 'div', 'div', 'button', 'div', 'div', 'button', 'div', 'div', 'button', 'div', 'div', 'button', 'div', 'button', 'svg', 'line', 'pol

In [87]:
len(tags), len(closures), len(contents), len(attributes)

(199, 199, 199, 199)

In [88]:
from termcolor import colored

In [89]:
for i in range(len(tags)):
    print('---'*50)
    print(f'open : {tags[i]}\t\tclose : {closures[i]}\t\tattributes : {attributes[i]}')
    print('content : ',colored(f'{contents[i]}','green'))

------------------------------------------------------------------------------------------------------------------------------------------------------
open : html		close : html		attributes : {}
content :  
------------------------------------------------------------------------------------------------------------------------------------------------------
open : head		close : head		attributes : {}
content :  
------------------------------------------------------------------------------------------------------------------------------------------------------
open : meta		close : meta		attributes : {'charset': 'utf-8'}
content :  
------------------------------------------------------------------------------------------------------------------------------------------------------
open : link		close : link		attributes : {'href': '/assets/images/placeholder.jpg', 'rel': ['shortcut', 'icon']}
content :  
-----------------------------------------------------------------------------------------